In [1]:
import scipy
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import scipy.optimize as spo
import scipy.constants as const
from scipy.spatial.transform import Rotation as R

## Constants

In [2]:
# Valor de bohr en metros
bohr_factor = const.value('Bohr radius')
# Factor de conversión de angstrom a bohr
angstrom_to_bohr = const.angstrom / bohr_factor
hartree_to_kcal = 627.509
kJ_to_kcal = 1 / 4.184

# optimization

In [3]:
from scipy import optimize

#### Functions for optimization or calculate differences

In [4]:
#estas son las coordenadas de quin para analizar cada posicion comparandolas con las de pf6 y sacando el resultado
coordinates_quin_to_pf6_1 = np.load (r"C:\Users\migue\Downloads\Pf6_quin_configurations_genetic.npy")
coordinates_quin_to_pf6_2 = np.load (r"C:\Users\migue\Downloads\quin_configurations_genetic.npy")

coordinates_quin_to_pf6_total = np.concatenate((coordinates_quin_to_pf6_1, coordinates_quin_to_pf6_2), axis = 0)


def xyz_a_matriz_numpy(ruta_archivo_xyz):
    # Leer el contenido del archivo
    with open(ruta_archivo_xyz, 'r') as archivo:
        lineas = archivo.readlines()

    # Omitir la primera línea (contiene el número total de átomos) y otras no necesarias
    # Asumimos que las coordenadas comienzan desde la tercera línea
    lineas_coordenadas = lineas[2:]
    
    # Preparar una lista para almacenar las coordenadas
    coordenadas = []
    
    for linea in lineas_coordenadas:
        if linea.strip():  # Verificar que la línea no esté vacía
            partes = linea.split()
            if len(partes) == 4:  # Asegurar que la línea tenga el formato esperado (Elemento x y z)
                # Convertir las partes numéricas (x, y, z) a flotantes y añadir a la lista
                coordenadas.append([float(partes[1]), float(partes[2]), float(partes[3])])

    # Convertir la lista de coordenadas a una matriz NumPy
    matriz_coordenadas = np.array(coordenadas)

    return matriz_coordenadas

# estas son las coordenadas de los pf6 para poder sacar las distancias, estas son todas sus configuraciones.
coord_pf6_80 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\PF6_centered_for_geneticAlg.xyz")
coord_pf6_18_1 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\pf6_centered_180_1_genetic.xyz")
coord_pf6_18_2 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\pf6_centered_180_2_genetic.xyz")
coord_pf6_18_3 = xyz_a_matriz_numpy(r"C:\Users\migue\Downloads\pf6_centered_180_3_genetic.xyz")


def space_distances_generation(base_molecule, configurations):
    space_distance_vec = []
    for w in range(len(configurations)):  # w is the distances vector
        space_distance = cdist(base_molecule, configurations[w], 'euclidean')
        space_distance = np.array(space_distance.flatten())
        space_distance_vec.append(space_distance)
    return np.array(space_distance_vec)

energies_80 = np.load(r"C:\Users\migue\Downloads\pf6_quin_energies_80.npz")
energies_180_1 = np.load(r"C:\Users\migue\Downloads\quin_pf6_180_1_ene.npz")
energies_180_2 = np.load(r"C:\Users\migue\Downloads\quin_fp6_180_2_ene.npz")
energies_180_3 = np.load(r"C:\Users\migue\Downloads\quin_pf6_180_3_ene.npz")

energies_pf6_80 = energies_80['esapt']
energies_pf6_180_1 = energies_180_1['esapt']*hartree_to_kcal
energies_pf6_180_2 = energies_180_2['esapt']*hartree_to_kcal
energies_pf6_180_3 = energies_180_3['esapt']*hartree_to_kcal

total_energies = np.array(np.concatenate((energies_pf6_80,energies_pf6_180_1,energies_pf6_180_2, energies_pf6_180_3), axis = 0))

distances_80 = space_distances_generation(coord_pf6_80, coordinates_quin_to_pf6_1)
distances_180_1 = space_distances_generation(coord_pf6_18_1, coordinates_quin_to_pf6_2)
distances_180_2 = space_distances_generation(coord_pf6_18_2, coordinates_quin_to_pf6_2)
distances_180_3 = space_distances_generation(coord_pf6_18_3, coordinates_quin_to_pf6_2)

total_distances = np.concatenate((distances_80, distances_180_1, distances_180_2, distances_180_3), axis = 0)


####concatenar distancias 

charges_chn = [0.038097, -0.167411, -0.167412, 0.030591, -0.013700, -0.013700, -0.234656,
           0.272792, 0.094346, 0.097032,  0.097032, 0.098344, 0.098344, 0.079640, 0.079641,
           0.092649, 0.092649, 0.091285, 0.091285, 0.103953, 0.103953, 0.035246]


charges_pf6 = [-0.547209,  -0.547209,  -0.547209,  -0.542063, -0.542063,
               -0.552955,  2.272009]


# a b c for molecules C(4), H(4), H(1), N(3), F, P
p_v = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                 2376.55, 191935, 3.48, 283248.0316091423, 273570.9941372, 4.0775266,
                 283248.0316091423, 273570.9941372, 4.0775266]

def vector_q1q2(vec_1, vec_2):
    v_q1q2 = np.outer(vec_1, vec_2).flatten()
    return np.array(v_q1q2)


vec_q1q2_pf6_quin = vector_q1q2(charges_pf6, charges_chn)

In [168]:
def function_test(solution):
    p_v = solution
    vec_1a = [p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[3],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[9]]
    vec_1b = [p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[4],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[10]]
    vec_1c = [p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[5],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[11]]
    vec_1a = np.array(vec_1a)
    vec_1b = np.array(vec_1b)
    vec_1c = np.array(vec_1c)
    vec_2a = [p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[15]]
    vec_2b = [p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[16]]
    vec_2c = [p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[17]]
    vec_2a = np.array(vec_2a)
    vec_2b = np.array(vec_2b)
    vec_2c = np.array(vec_2c)

    v_12a = np.array((np.outer(vec_1a, vec_2a).flatten())**(0.5))
    v_12b = np.array((np.outer(vec_1b, vec_2b).flatten())**(0.5))
    v_12c = np.array(((0.5)*(vec_1c[:, np.newaxis] + vec_2c)).flatten())

    vec_d = total_distances ## desde fuera
    vec_a, vec_b, vec_c = v_12a, v_12b, v_12c
    vec_q1q2 = vec_q1q2_pf6_quin ### desde fuera
    
   

    combination_matrix = []
    for vector in vec_d:
        combination_matrix.append(np.vstack((vector, vec_a, vec_b, vec_c, vec_q1q2)))

    combination_matrix = np.array(combination_matrix)

    #### Transformar las energias objetivo (recordatorio mio xd)

    energies = []
    for i in range(combination_matrix.shape[0]):
        sum = 0
        for j in range(combination_matrix.shape[2]):
            exp_term = (combination_matrix[i, 2, j])*(np.exp((-combination_matrix[i, 3, j])*(combination_matrix[i, 0, j])))
            dis_ind_term = combination_matrix[i, 1, j]/((combination_matrix[i, 0, j])**6)
            elst_term = (combination_matrix[i, 4, j])/(combination_matrix[i, 0, j]*angstrom_to_bohr)
            sum = sum + (exp_term*kJ_to_kcal) - (dis_ind_term*kJ_to_kcal) + (elst_term*hartree_to_kcal)
        energies.append(sum)

    penalization_vec = penalization_funtion_2(solution)
    diferencias = energies - total_energies
    diferencias_final = np.hstack((np.array(diferencias), np.array(penalization_vec))) # + (np.array(diferencias) * penalization_funtion(solution))
    #diferencias = diferencias.sum()

    return  np.sum(diferencias**2)#_final**2) ## np.sum(np.abs(diferencias))

In [87]:
x = function_test([9.420360000e+02, 1.200000e+04, 3.30000000e+00, 300.00000000e+00,
  1.4185747e+03, 3.43000000e+00, 2.76370000e+02, 6.7052353e+03,
  3.1777550e+00, 1.70791554e+03, 1.9065018e+05, 3.37316262e+00,
  3.210003161e+03, 1.1809078e+05, 4.20752660e+00, 3.30500003161e+03,
  1.302297698e+05, 4.1252660e+00])
print(x)

333.44808691518097


In [244]:
def penalization_funtion(x):
    x = np.array(x)
    penalty = [] 
    original_parameters = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                           2376.55, 191935, 3.48, 1725.2045067499794, 803364.870629, 6.14391455469,
                           4591.876112601954, 126176948.8657, 3.250978952232]
    threshold = .10  #outside the threshold
    for id in range(len(x)):
        if original_parameters[id] == 0.00 :
            penalty.append(x[id])
        else: 
            percentage =  np.abs((original_parameters[id] - x[id])/(original_parameters[id]))
            if percentage > threshold :
                penalty.append(((percentage - threshold)*100)**2) 
            else: 
                penalty.append(0)
    

    p_v = penalty
    

    vec_1a = [p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[3],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[9]]
    vec_1b = [p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[4],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[10]]
    vec_1c = [p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[5],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[11]]
    vec_1a = np.array(vec_1a)
    vec_1b = np.array(vec_1b)
    vec_1c = np.array(vec_1c)
    vec_2a = [p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[15]]
    vec_2b = [p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[16]]
    vec_2c = [p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[17]]
    vec_2a = np.array(vec_2a)
    vec_2b = np.array(vec_2b)
    vec_2c = np.array(vec_2c)

    v_12a = np.array((vec_1a[:, np.newaxis] + vec_2a).flatten())
    v_12b = np.array((vec_1b[:, np.newaxis] + vec_2b).flatten())
    v_12c = np.array((vec_1c[:, np.newaxis] + vec_2c).flatten())
    
    v_penalization =  v_12a + v_12b + v_12c 

    
    return v_penalization.sum()/154

In [245]:
def penalization_funtion_2(x):
    x = np.array(x)
    penalty = [] 

    original_parameters = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                           2376.55, 191935, 3.48, 1725.2045067499794, 275000, 4.1,
                           4591.876112601954, 275000.00, 4.1]

    threshold = .2  #outside the threshold
    for id in range(len(x)):

        if original_parameters[id] == 0.00 :
            penalty.append(x[id])
        else: 
            percentage =  np.abs((original_parameters[id] - x[id])/(original_parameters[id]))
            if percentage > threshold :
                penalty.append(percentage*1000) # 10 100 
            else: 
                penalty.append(0)
    return penalty
    


In [243]:
h = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                           2376.55, 191935, 3.48, 1725.2045067499794*2, 803364.870629, 6.14391455469,
                           4591.876112601954*2, 1261769.8657, 3.250978952232]

In [155]:
x = penalization_funtion_2(h)
print(x)

[0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
resultados =  function_test([978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                 2376.55, 191935, 3.48, 824.667546247, 803364.870629, 6.14391455469,
                 2194.9694610908, 126176948.8657, 3.250978952232])

print(resultados)

2360999.5607572636


Para probar los resultados con los valores que ya tenia optimos de la optimizacion pasada

In [50]:
resultados2 = function_test([978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                 2376.55, 191935, 3.48, 1725.2045067499794, 803364.870629, 6.14391455469,
                 4591.876112601954, 126176948.8657, 3.250978952232])
print(resultados2)

4356.5556810029275


In [7]:
resultados_opt = function_test([9.420360000e+02, 1.200000e+04, 3.30000000e+00, 300.00000000e+00,
  1.4185747e+03, 3.43000000e+00, 2.76370000e+02, 6.7052353e+03,
  3.1777550e+00, 1.70791554e+03, 1.9065018e+05, 3.37316262e+00,
  3.210003161e+03, 1.1809078e+05, 4.20752660e+00, 3.30500003161e+03,
  1.302297698e+05, 4.1252660e+00])

print(resultados_opt)

65.58642608762905


Energies results

In [8]:
x_start = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                 2376.55, 191935, 3.48, 1725.2045067499794, 803364.870629, 6.14391455469,
                 4591.876112601954, 126176948.8657, 3.250978952232]

x_start = [9.420360000e+02, 1.200000e+04, 3.30000000e+00, 300.00000000e+00,
  1.4185747e+03, 3.43000000e+00, 2.76370000e+02, 6.7052353e+03,
  3.1777550e+00, 1.70791554e+03, 1.9065018e+05, 3.37316262e+00,
  3.210003161e+03, 1.1809078e+05, 4.20752660e+00, 3.30500003161e+03,
  1.302297698e+05, 4.1252660e+00]

In [214]:
start_new = [978.36, 131571, 3.60, 0.00, 764.9, 3.56, 278.37, 12680, 3.56,
                           2376.55, 191935, 3.48, 1725.2045067499794, 275000, 4.1,
                           4591.876112601954, 275000.00, 4.1]

In [246]:
bounds = [(0, None) for _ in x_start]
# bounds = [(0, None)] * len(x_start)

result = spo.minimize(function_test, h, options={"disp": True}, bounds=bounds)


In [247]:
print(result)

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 1020819.5893117375
        x: [ 9.784e+02  1.316e+05 ...  1.262e+06  3.901e+00]
      nit: 6
      jac: [ 1.513e-01  0.000e+00 ...  0.000e+00 -1.595e+03]
     nfev: 1482
     njev: 78
 hess_inv: <18x18 LbfgsInvHessProduct with dtype=float64>


In [248]:
optimized_parameters = result.x
print("Parámetros optimizados:", optimized_parameters)

Parámetros optimizados: [9.78359994e+02 1.31571000e+05 3.63255133e+00 0.00000000e+00
 7.64900000e+02 3.55999584e+00 2.78370369e+02 1.26800000e+04
 4.15752421e+00 2.37655000e+03 1.91935000e+05 3.47955391e+00
 3.45040901e+03 8.03364871e+05 6.12479603e+00 9.18375224e+03
 1.26176987e+06 3.90117162e+00]


### Parameters check with all the data

In [14]:
def check(solution):
    p_v = solution
    vec_1a = [p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[0],p_v[3],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[6],p_v[9]]
    vec_1b = [p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[1],p_v[4],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[7],p_v[10]]
    vec_1c = [p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[2],p_v[5],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[8],p_v[11]]
    vec_1a = np.array(vec_1a)
    vec_1b = np.array(vec_1b)
    vec_1c = np.array(vec_1c)
    vec_2a = [p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[12],p_v[15]]
    vec_2b = [p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[13],p_v[16]]
    vec_2c = [p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[14],p_v[17]]
    vec_2a = np.array(vec_2a)
    vec_2b = np.array(vec_2b)
    vec_2c = np.array(vec_2c)

    v_12a = np.array((np.outer(vec_1a, vec_2a).flatten())**(0.5))
    v_12b = np.array((np.outer(vec_1b, vec_2b).flatten())**(0.5))
    v_12c = np.array(((0.5)*(vec_1c[:, np.newaxis] + vec_2c)).flatten())

    vec_d = total_distances ## desde fuera
    vec_a, vec_b, vec_c = v_12a, v_12b, v_12c
    vec_q1q2 = vec_q1q2_pf6_quin ### desde fuera
    
   

    combination_matrix = []
    for vector in vec_d:
        combination_matrix.append(np.vstack((vector, vec_a, vec_b, vec_c, vec_q1q2)))

    combination_matrix = np.array(combination_matrix)

    #### Transformar las energias objetivo (recordatorio mio xd)

    energies = []
    for i in range(combination_matrix.shape[0]):
        sum = 0
        for j in range(combination_matrix.shape[2]):
            exp_term = (combination_matrix[i, 2, j])*(np.exp((-combination_matrix[i, 3, j])*(combination_matrix[i, 0, j])))
            dis_ind_term = combination_matrix[i, 1, j]/((combination_matrix[i, 0, j])**6)
            elst_term = (combination_matrix[i, 4, j])/(combination_matrix[i, 0, j]*angstrom_to_bohr)
            sum = sum + (exp_term*kJ_to_kcal) - (dis_ind_term*kJ_to_kcal) + (elst_term*hartree_to_kcal)
        energies.append(sum)
        
    diferencias = energies - total_energies
    diferencias = np.array(diferencias)
    diferencias = diferencias.sum()

    return diferencias

In [238]:
values_optimized = check(optimized_parameters)
print(values_optimized)

67.24370039571568


Values actualization

In [249]:
def actualizar_coeficientes(atom_types, vector):
    elementos = ['C', 'HN', 'H', 'N', 'F', 'P']
    variables = ['C', 'A', 'B']

    # Itera sobre cada elemento y variable en el vector y actualiza el diccionario
    for i, elemento in enumerate(elementos):
        for j, variable in enumerate(variables):
            # Calcula el índice correspondiente en el vector para acceder al valor
            indice = i * len(variables) + j
            # Actualiza el valor en el diccionario
            atom_types[elemento][variable] = vector[indice]

# Ejemplo de uso
atom_types = {
    'F': {'mass': 18.998403, 'charge': -0.17, 'A': 118090.78, 'B': 4.20752660, 'C': 3210.003161},
    'P': {'mass': 30.973762, 'charge': 1 , 'A': 130229.7698, 'B': 4.1252660, 'C': 3305.00003161},
    'C': {'mass': 12.011000, 'charge': - 0.1115 , 'A': 12000.00, 'B': 3.3000, 'C': 942.0360000},
    'H': {'mass': 1.008000 , 'charge': 0.1150, 'A': 6705.2353, 'B': 3.1777550, 'C': 276.37000},
    'HN': {'mass': 1.008000 , 'charge': 0.1150, 'A': 1418.5747, 'B': 3.43000, 'C': 300.00000000},
    'N': {'mass': 14.007000 , 'charge': 0.5973, 'A': 190650.18, 'B': 3.37316262, 'C': 1707.91554}
}

vector_valores = optimized_parameters

# Actualizar los coeficientes en atom_types usando el vector de valores
actualizar_coeficientes(atom_types, vector_valores)

# Verificar los resultados
for elemento, valores in atom_types.items():
    print(elemento, valores)

print(atom_types)

F {'mass': 18.998403, 'charge': -0.17, 'A': 803364.870629, 'B': 6.1247960311413365, 'C': 3450.409005676946}
P {'mass': 30.973762, 'charge': 1, 'A': 1261769.8657012093, 'B': 3.901171624427035, 'C': 9183.752239463935}
C {'mass': 12.011, 'charge': -0.1115, 'A': 131571.0, 'B': 3.6325513256281408, 'C': 978.3599937465764}
H {'mass': 1.008, 'charge': 0.115, 'A': 12679.999971626705, 'B': 4.157524213899933, 'C': 278.37036881531316}
HN {'mass': 1.008, 'charge': 0.115, 'A': 764.9, 'B': 3.559995842190495, 'C': 0.0}
N {'mass': 14.007, 'charge': 0.5973, 'A': 191935.0, 'B': 3.4795539088936565, 'C': 2376.5499986827717}
{'F': {'mass': 18.998403, 'charge': -0.17, 'A': 803364.870629, 'B': 6.1247960311413365, 'C': 3450.409005676946}, 'P': {'mass': 30.973762, 'charge': 1, 'A': 1261769.8657012093, 'B': 3.901171624427035, 'C': 9183.752239463935}, 'C': {'mass': 12.011, 'charge': -0.1115, 'A': 131571.0, 'B': 3.6325513256281408, 'C': 978.3599937465764}, 'H': {'mass': 1.008, 'charge': 0.115, 'A': 12679.999971626

In [192]:
w = function_test(h)
w 

2334116.0602799994

In [174]:
w = function_test([9.420360000e+02, 1.200000e+04, 3.30000000e+00, 300.00000000e+00,
  1.4185747e+03, 3.43000000e+00, 2.76370000e+02, 6.7052353e+03,
  3.1777550e+00, 1.70791554e+03, 1.9065018e+05, 3.37316262e+00,
  3.210003161e+03, 1.1809078e+05, 4.20752660e+00, 3.30500003161e+03,
  1.302297698e+05, 4.1252660e+00])
w 

10407238.514492482